In [3]:
# doing an xg model based on the features extracted using MLlib
# shot_statsbomb_xg is the target variable, the rest are the features
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor, DecisionTreeRegressor, GeneralizedLinearRegression, IsotonicRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
import pp_events as pp
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('Training').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 16:19:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/21 16:19:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# xgboost model
from xgboost import XGBRegressor
# XGBoost, LightGBM, CatBoost
#from pyspark.ml.classification import MultilayerPerceptronClassifier
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
events = spark.read.csv('Data/events.csv',header=True,inferSchema=True,sep=';')

In [ ]:
events_shot = events.filter(events.type=='Shot')

events_shot = pp.split_location(events_shot)
events_shot = pp.distance_to_goal(events_shot)
events_shot = pp.get_shot_angle(events_shot)

In [ ]:
features = ["shot_location_x", "shot_location_y", "distance_to_goal", "shot_angle"]
assembler = VectorAssembler(inputCols=features, outputCol="features")
data = assembler.transform(events_shot).select("features", "shot_statsbomb_xg")

# Split into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Initialize all models
lr = LinearRegression(labelCol="shot_statsbomb_xg", featuresCol="features")
rf = RandomForestRegressor(labelCol="shot_statsbomb_xg", featuresCol="features")
gbt = GBTRegressor(labelCol="shot_statsbomb_xg", featuresCol="features")
dt = DecisionTreeRegressor(labelCol="shot_statsbomb_xg", featuresCol="features")
glr = GeneralizedLinearRegression(labelCol="shot_statsbomb_xg", featuresCol="features")
ir = IsotonicRegression(labelCol="shot_statsbomb_xg", featuresCol="features")

# Create a list of models
models = [lr, rf, gbt, dt, glr, ir]

# Create a list of model names
model_names = ["Linear Regression", "Random Forest", "Gradient Boosted Trees", "Decision Tree", "Generalized Linear Regression", "Isotonic Regression"]

# Create a list of model pipelines
model_pipelines = [Pipeline(stages=[model]) for model in models]

In [ ]:
# Fit each model pipeline
fitted_models = [pipeline.fit(train_data) for pipeline in model_pipelines]

In [ ]:
# Evaluate each model
evaluator = RegressionEvaluator(labelCol="shot_statsbomb_xg", predictionCol="prediction", metricName="rmse")
predictions = [fitted_model.transform(test_data) for fitted_model in fitted_models]
rmse = [evaluator.evaluate(prediction) for prediction in predictions]

In [ ]:
# Display the RMSE of each model
for i in range(len(models)):
    print(model_names[i] + " RMSE: " + str(rmse[i]))

In [ ]:
# Time spent to run : 1m 54s for ["shot_location_x", "shot_location_y", "distance_to_goal", "shot_angle"] 0.09
# Time spent to run : 1m 50s for ["distance_to_goal", "shot_angle"]                                       0.10

# Consider using Classification instead of Regression